# Keep intersect of samples across the 50 rMATS final matrices

## 0. Loading dependencies

In [ ]:
# util R packages
library(readr)
library(data.table)
library(R.utils) # required for data.table's gzip util

In [ ]:
# set parameters
rmats_matrices_suffix <- "*.txt.gz"

## 1. Access and decompress the `rmats_final.tar.gz` 
The successfully completed Nextflow analysis, that has combined all individual samples matrices into 50 rMATS final matrices utilises the [create_matrices_from_files.sh](https://github.com/lifebit-ai/rmats-nf/blob/master/containers/post-rmats/create_matrices_from_files.sh). This analysis has generated a single archive file named `rmats_final.tar.gz` that contains all the 50 summarised matrices, in `.txt.gz` format.

We can access the `rmats_final.tar.gz` from the respective google project bucket in the location:

<img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/google_cloud_logo.png" width="30"/>   **`  gs://robinson-bucket/results/merged_matrices/rmats_final.tar.gz`**

On CloudOS, we can link the data from the bucket in the Jupyter Notebook session. For example, if we would like to access the data from a successfully completed Nextflow job, in this example the `rmats_final.tar.gz`, we can follow the steps below:

|STEP|DESCRIPTION|
|:---|:---|
|1. | Go to the **Analysis Job page** of the pipeline with the results of interest <a href="https://cloudos.lifebit.ai/app/jobs/5e83b03ee7d1990104ccacd4" target="_blank"><img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_badge.png" width="80"/></a> |c
|2. | Click on the  <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_results.png" width="65"/> tab and copy the results folder name from  <kbd>rMATS-batches > **merge-rmats-5e83b03ee7d1990104ccacd4** </kbd> (project > results folder) |
| 3.| <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_jupyter_grey_icon.png" width="25"/>  Navigate to the Jupyter Notebook Session in CloudOS |
| 4.| Create a new Jupyter Notebook Session by clicking <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_new_jupyter_session.png" width="55"/> ; it will take 1-2' until your sesssion is initialised |
| 5. | Click on the blue arrow on the left of the screen <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_jupyter_add_data.png" width="25"/> to expand the **`Session data`** <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_add_dataset.png" width="25"/> interface |
|5. |Once the `Data` interface pops up, click on **`Project results`** |
|6. | Select the project associated with the Nextflow job, ie. `rMATS-batches`|
|7. | For convenience, <kbd>**CTRL + F**</kbd> the folder results name you copied in <u>step 2</u> and select this folder |
|8. | Navigate inside the results folder, select the `rmats_final.tar.gz` file and click the choose button <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_data_choose.png" width="55"/>  to initiate the data mounting will initiate
|9. |  Once the loader icon stops spinning your data are available to use <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/cloudos_jupyter_mounted_data_loaded.gif" width="185"/>  
|10. | Your imported data will appear in the folder named `mounted_data/`. You can access the data: **i)** by clicking the JupyterLab parent folder icon on the top left of the screen <img src="https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/jupyter_lab_parent_folder.png " width="35"/> and navigating into the folders or **ii)** using the terminal `cd /mnt/shared/gcp-user/session_data/mounted_data/`


**Here is an overview of all the steps described above:**
---

![](https://github.com/TheJacksonLaboratory/sbas/raw/master/assets/access_data.gif)


> **NOTE**: To work with shorter filepaths, we can move the `rmats_final.tar.gz` file in the `data` folder, which we use as a vessel for all input files during the analysis. 

In [ ]:
longpath <-  "../../mounted-data/cloudosinputdata/deploit/teams/5e601f9f786cca0107921955/users/5c066455e0744f007585e826/projects/5e820cb0bc45f60104372e9a/jobs/5e83b03ee7d1990104ccacd4/results/results/archive/rmats_final.tar.gz"
system(paste0("mv ", longpath, " ../data/"), intern = TRUE)

We proceed to decompress the archive to access the 50 matrices:

In [ ]:
system("tar -xvzf ../data/rmats_final.tar.gz", intern = TRUE)

## 2. Delete the `*txt.filelist.txt` files that are not required

In [ ]:
system("rm -f ../data/*txt.filelist.txt", intern = TRUE)

## 3. Collect the samples names per file for each one of the 50 rMATS final matrices

We will use a bash for loop to write the header with the samples' SRA ids in a single column file named `<initial_rmats_matrix>.samples.csv`. After running the following command we will have 50 single column files.

The command is the following
([detailed explanation here](https://explainshell.com/explain?cmd=for+entry+in+%60ls+*txt%60%3B+do+echo+%24entry+%3E%3E+%24%7Bentry%7D.samples.csv+%26%26+head+-1+%24entry+%7C+sed+%27s%2F%2C%2F%5C%27%24%27%5Cn%27%27%2Fg%27+%7C+grep+-v+ID+%3E%3E+%24%7Bentry%7D.samples.csv%3B+done))

```bash
for entry in `ls *txt`; do echo $entry >> ${entry}.samples.csv && head -1 $entry | sed 's/,/\'$'\n''/g' | grep -v ID >> ${entry}.samples.csv; done

```

- For every file with a *txt suffix:
```bash 
for entry in `ls *txt` 
```

- Print the filename,<br> 
as a first row in a filenamed **${entry}.samples.csv**
(This will be the header of the single column samples id file) <br>

- Print the whole first row of the file (the header with all sample ids)

```bash
do echo $entry >> ${entry}.samples.csv && head -1 $entry 
```

- pipe this into sed, to replace the `\t` with `\n` to create a 1 column file


```bash
 sed 's/,/\'$'\n''/g'
```

- pipe this into `grep -v` to omit the `ID` colname 

```bash
grep -v ID
```

We wrote the command in a script and now we can run it with bash:

In [ ]:
system("bash collect_sample_names.sh", intern=TRUE)

**NOTE**

We will also `gzip` the rmats matrices for saving space in our device. This step takes a long time as the file sizes are considerably large.

In [ ]:
system("gzip *txt", intern = TRUE)

## 4. Use `paste` in `bash` to combine the 50 single column files into one metadata .csv file

The `colnames` will be the matrices names and each column will have SRA ids as entries.

The command: 

```bash
paste -d',' *.samples.csv > samples.csv
```

In [ ]:
system("paste -d',' *.samples.csv > samples.csv", intern = TRUE)

Let's now inspect the created dataframe:|

In [ ]:
samples <- read.csv("samples.csv", sep = ",")

In [ ]:
tail(samples)
message(paste0("\nmatrix dimensions:  ",paste0(dim(samples), collapse = " , ")))

Now we can delete the single column files:


In [ ]:
system("rm -f *.txt.samples.csv", intern = TRUE)

## 5. Find out what are the sample ids present in all 50 matrices
In order to do that, we will collect all the SRA ids present in the collective table in a vector and count the frequency of the samples. We will select only the ids that have a frequency in the vector equalt to 50.

In [ ]:
all <- readr::read_csv("samples.csv", col_names = TRUE)
all_counts       <- as.vector(t(all))
all_counts_df    <- as.data.frame(table(all_counts))

In [ ]:
head(all_counts_df, 2)

Now we will create a vector named `toKeep` to use it for filtering the dataframe. We will select from each one of the rMATS final matrices only the columns which column names are included in the `toKeep` list. We are also adding in the collection the first column named `'ID'` which we omited including in the <rmats-file>.sample.csv matrices.

In [ ]:
toKeep   <- c("ID", as.vector(all_counts_df$all_counts[(all_counts_df$Freq==50)]))
toRemove <- c(as.vector(all_counts_df$all_counts[(all_counts_df$Freq!=50)]))
message(paste0("length(toKeep): " , length(toKeep) ))
message(paste0("length(toRemove): ", length(toRemove) ))

## 6. Apply the filtering to all 50 matrices
We loop over all rMATS files to select from each table only the columns in the intersect of samples.

In [ ]:
# create a dir to store the filtered matrices named intersect if it doesn't exist -p options
system("mkdir -pv intersect", intern = TRUE)

In [ ]:
my_files <- list.files(recursive = FALSE, pattern = rmats_matrices_suffix)
length(my_files)
head(my_files)

We will run below a forloop that will

1. read with the efficient [`{data.table}` R package](https://csgillespie.github.io/efficientR/data-processing-with-data-table.html) each of the `.txt.gzip` rMATS matrices files
2. filter and keep only the SRA samples that belong to the intersect of the sample ids across the 50 matrices
3. write the updated filtered rMATS table directly in a compressed `.txt.gz` file in the folder named `intersect`
4. delete the initial respective matrix

> **NOTE**: The `{data.table}` R package is very efficient, especially for handling files with many columns. Here's a benchmark from the "Efficient R programming" book[$^{1}$](https://csgillespie.github.io/efficientR/) from Colin Gillespie and Robin Lovelace:

 
 <center><a href="https://csgillespie.github.io/efficientR/data-processing-with-data-table.html" target="_blank"><img src="https://csgillespie.github.io/efficientR/_main_files/figure-html/dtplot-1.png"  width="400" align="center" /></a></center>
 


Let's start the for loop and count time of execution:

In [ ]:
# Start the clock!
start_time <- Sys.time()

for (i in 1:length(my_files)) {
    filename <- my_files[[i]]
    # The data.table R package can read/write with gzip compression
    rmats              <- data.table::fread(filename, sep = ",")

    # with = FALSE is necessary, follows syntax idiom of the data.table package
    rmats_filtered     <- rmats[ , toKeep, with=FALSE]

    data.table::fwrite(rmats_filtered, 
                     file = paste0("intersect/", filename), 
                     sep = ",", 
                     compress = "gzip",
                     col.names = TRUE)

    # Remove the objects after the file is written
    message(paste0(i, ". '", filename, "' updated"), appendLF = FALSE)  
    system(paste0("rm " , filename), intern = TRUE)
    rm(filename, rmats, rmats_filtered)

}
end_time <- Sys.time()
end_time - start_time
